In [1]:
from pyspark import SparkContext
from datetime import datetime
import random

In [2]:
sc = SparkContext(appName="Movie plots")

In [3]:
metadata = sc.textFile("movie.metadata.tsv")

In [4]:
k_gram_size=10

In [5]:
 metadata = metadata.map(lambda line: line.split("\t")).map(lambda a: (int(a[0]),a[2]))

In [6]:
metadata.take(10)

[(975900, 'Ghosts of Mars'),
 (3196793, 'Getting Away with Murder: The JonBenét Ramsey Mystery'),
 (28463795, 'Brun bitter'),
 (9363483, 'White Of The Eye'),
 (261236, 'A Woman in Flames'),
 (13696889, 'The Gangsters'),
 (18998739, "The Sorcerer's Apprentice"),
 (10408933, "Alexander's Ragtime Band"),
 (9997961, 'Contigo y aquí'),
 (2345652, 'City of the Dead')]

In [7]:
movie_plots = sc.textFile("plot_summaries.txt")

In [8]:
movie_plots = movie_plots.map(lambda line: line.split("\t")).map(lambda a: (int(a[0]),a[1]))

In [9]:
movie_plots.take(1)

[(23890098,
  "Shlykov, a hard-working taxi driver and Lyosha, a saxophonist, develop a bizarre love-hate relationship, and despite their prejudices, realize they aren't so different after all.")]

In [10]:
#shingle_set = {}

In [11]:
def break_k_grams(plot):
    shingles = []
    for i in range(len(plot)-k_gram_size+1):
        shingles.append(plot[i:i+k_gram_size])
        #if 
    return shingles

In [12]:
movie_shingles = movie_plots.mapValues(break_k_grams)

In [13]:
movie_shingles.take(1)

[(23890098,
  ['Shlykov, a',
   'hlykov, a ',
   'lykov, a h',
   'ykov, a ha',
   'kov, a har',
   'ov, a hard',
   'v, a hard-',
   ', a hard-w',
   ' a hard-wo',
   'a hard-wor',
   ' hard-work',
   'hard-worki',
   'ard-workin',
   'rd-working',
   'd-working ',
   '-working t',
   'working ta',
   'orking tax',
   'rking taxi',
   'king taxi ',
   'ing taxi d',
   'ng taxi dr',
   'g taxi dri',
   ' taxi driv',
   'taxi drive',
   'axi driver',
   'xi driver ',
   'i driver a',
   ' driver an',
   'driver and',
   'river and ',
   'iver and L',
   'ver and Ly',
   'er and Lyo',
   'r and Lyos',
   ' and Lyosh',
   'and Lyosha',
   'nd Lyosha,',
   'd Lyosha, ',
   ' Lyosha, a',
   'Lyosha, a ',
   'yosha, a s',
   'osha, a sa',
   'sha, a sax',
   'ha, a saxo',
   'a, a saxop',
   ', a saxoph',
   ' a saxopho',
   'a saxophon',
   ' saxophoni',
   'saxophonis',
   'axophonist',
   'xophonist,',
   'ophonist, ',
   'phonist, d',
   'honist, de',
   'onist, dev',
   'nist, deve',
  

In [14]:
#optional but advised hashing of shingles into 4 bytes (%2**4*8)
movie_shingles = movie_shingles.mapValues(lambda shing: [hash(x)%(2**32) for x in shing])
movie_shingles.take(1)

[(23890098,
  [94532052,
   1333049197,
   3672963186,
   1726631474,
   1863322425,
   614526316,
   1894335871,
   2689497919,
   2080848502,
   4174466113,
   369837028,
   3214026721,
   3945506424,
   1747605881,
   2783439332,
   1401102472,
   1739839766,
   1715324274,
   2298733938,
   2714408454,
   213558704,
   343706503,
   1442681580,
   3148770810,
   3241239299,
   1752129976,
   144130560,
   3086576788,
   160637640,
   3269353480,
   1339222350,
   1286296005,
   561837381,
   1236085361,
   3858618623,
   3934138345,
   1689705407,
   1350957499,
   2593179710,
   3994009040,
   2289160871,
   534521501,
   3118263062,
   2036980624,
   2393102961,
   3723656755,
   771509250,
   3447800312,
   3074807925,
   2606892755,
   4038614307,
   3421457674,
   2941274790,
   1043505956,
   523873563,
   2735068347,
   2731632074,
   2265473228,
   2745059992,
   3129550152,
   4189888240,
   4273308920,
   2817425012,
   3900401986,
   3252307003,
   2610653454,
   3021585

In [15]:
def random_hash():
    a = random.randint(0,1000)
    b = random.randint(0,1000)
    return a,b

In [16]:
random_hash()

(367, 389)

In [17]:
n_shingles = 23254807 #movie_shingles.flatMap(lambda tup: tup[1]).distinct().count()

In [22]:
#prime number for hashing
p = 24000001
#number of hash functions
k=100
hash_functions = []
for i in range(k):
    hash_functions.append(random_hash())
hash_functions

[(91, 310),
 (194, 58),
 (281, 70),
 (102, 80),
 (61, 113),
 (651, 670),
 (856, 645),
 (237, 244),
 (489, 228),
 (235, 266),
 (614, 121),
 (544, 856),
 (619, 167),
 (703, 970),
 (980, 406),
 (28, 248),
 (989, 711),
 (460, 879),
 (28, 714),
 (279, 464),
 (280, 897),
 (659, 504),
 (535, 397),
 (572, 59),
 (919, 268),
 (687, 939),
 (654, 172),
 (741, 815),
 (633, 50),
 (298, 469),
 (818, 78),
 (943, 677),
 (165, 7),
 (494, 239),
 (738, 104),
 (498, 851),
 (812, 214),
 (1000, 460),
 (119, 698),
 (208, 246),
 (177, 214),
 (416, 206),
 (605, 457),
 (585, 860),
 (264, 551),
 (255, 37),
 (880, 104),
 (929, 440),
 (848, 104),
 (762, 901),
 (76, 564),
 (57, 448),
 (958, 738),
 (393, 766),
 (381, 507),
 (449, 994),
 (834, 597),
 (69, 204),
 (988, 337),
 (78, 992),
 (176, 83),
 (981, 318),
 (844, 537),
 (136, 308),
 (373, 445),
 (750, 68),
 (803, 220),
 (262, 584),
 (462, 237),
 (0, 300),
 (253, 977),
 (200, 176),
 (70, 862),
 (487, 33),
 (858, 954),
 (488, 580),
 (8, 627),
 (589, 494),
 (250, 969

In [23]:
def get_signature(hashed_shingles):
    signature_matrix = {}
    for element in hashed_shingles:
        for i in range(k):
            hash_result = calc_hash(*hash_functions[i],p,n_shingles, element)
            if  hash_result < signature_matrix.get(i,float('inf')):
                signature_matrix[i]=hash_result
    return signature_matrix

In [24]:
def calc_hash(a,b,prime,n,value):
    return ((a*value + b)%prime)%n
    

In [25]:
movie_shingles.mapValues(get_signature).take(2)

[(23890098,
  {0: 76187,
   1: 147087,
   2: 57835,
   3: 102314,
   4: 57881,
   5: 10212,
   6: 49907,
   7: 32808,
   8: 9413,
   9: 63915,
   10: 16216,
   11: 46069,
   12: 67876,
   13: 30757,
   14: 33793,
   15: 107540,
   16: 9331,
   17: 7122,
   18: 108006,
   19: 50582,
   20: 109700,
   21: 224056,
   22: 13935,
   23: 23362,
   24: 72884,
   25: 1092,
   26: 24108,
   27: 134817,
   28: 36704,
   29: 146324,
   30: 120434,
   31: 23926,
   32: 18631,
   33: 20770,
   34: 76035,
   35: 81121,
   36: 203249,
   37: 10569,
   38: 48791,
   39: 212088,
   40: 55697,
   41: 262556,
   42: 138268,
   43: 145815,
   44: 153781,
   45: 58356,
   46: 74647,
   47: 60806,
   48: 10286,
   49: 40080,
   50: 8466,
   51: 360120,
   52: 71433,
   53: 185584,
   54: 75843,
   55: 57279,
   56: 25008,
   57: 4036,
   58: 41399,
   59: 24100,
   60: 223545,
   61: 36222,
   62: 1752,
   63: 136620,
   64: 74108,
   65: 49134,
   66: 6600,
   67: 110560,
   68: 81340,
   69: 300,
   70: 3